In [1]:
import gym
import random
import numpy as np
import time
from gym.envs.registration import register
from IPython.display import clear_output

In [2]:
try:
    register(
        id='FrozenLakeNoSlip-v0',
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name': '4x4', 'is_slippery': False},
        max_episode_steps=100,
        reward_threshold=0.78, # optimum = .8196
    )
except:
    pass

In [3]:
env_name = 'FrozenLake-v0'
env_name = 'FrozenLakeNoSlip-v0'
env = gym.make(env_name)
print("Observation Space: ", env.observation_space)
print("Action Space: ", env.action_space)
type(env.action_space)

Observation Space:  Discrete(16)
Action Space:  Discrete(4)


gym.spaces.discrete.Discrete

In [4]:
class Agent():
    def __init__(self, env):
        self.is_discrete = \
            type(env.action_space) == gym.spaces.discrete.Discrete
        
        if self.is_discrete:
            self.action_size = env.action_space.n
            print("Action size: ", self.action_size)
        else:
            self.action_low = env.action_space.low
            self.action_high = env.action_space.high
            self.action_shape = env.action_space.shape
            print("Action range: ", self.action_low, self.action_high)
            
    def get_action(self, state):
        if self.is_discrete:
            action = random.choice(range(self.action_size))
        else:
            action = np.random.uniform(self.action_low,
                                       self.action_high,
                                       self.action_shape)
            
        return action

In [5]:
class QAgent(Agent):
    def __init__(self, env, discount_rate=0.97, learning_rate=0.01):
        super().__init__(env)
        self.state_size = env.observation_space.n
        print("State Size:", self.state_size)
        
        self.eps = 1.0
        self.discount_rate = discount_rate
        self.learning_rate = learning_rate
        self.build_model()
        
    def build_model(self):
        self.q_table = 1e-4*np.random.random([self.state_size, self.action_size])
    
    def get_action(self, state):
        q_state = self.q_table[state]
        action_greedy = np.argmax(q_state)
        action_random = super().get_action(state)
        return action_random if random.random() < self.eps else action_greedy
    
    def train(self, experience):
        state, action, next_state, reward, done = experience
        
        q_next = self.q_table[next_state]
        q_next = np.zeros([self.action_size]) if done else q_next
        q_target = reward + self.discount_rate * np.max(q_next)
        
        q_update = q_target - self.q_table[state, action]
        self.q_table[state, action] += self.learning_rate * q_update
        
        if done:
            self.eps = self.eps * 0.99

In [6]:
agent = QAgent(env)

total_reward = 0
for ep in range(100):
    state = env.reset()
    done = False
    while not done:
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)
        agent.train((state, action, next_state, reward, done))
        state = next_state
        total_reward += reward
        
        print("s:", state, "a:", action)
        print("ep:", ep, "tot_rew:", total_reward, "eps:", agent.eps)
        env.render()
        print(agent.q_table)
        time.sleep(.02)
        clear_output(wait=True)

s: 3 a: 2
ep: 18 tot_rew: 0.0 eps: 0.8345137614500874
  (Right)
SFFF
FHFH
FFFH
HFFG
[[3.11117096e-05 3.89985620e-05 9.43342424e-05 9.63468714e-05]
 [7.71907558e-05 5.62110064e-06 2.40749840e-05 5.58216342e-05]
 [2.58243487e-05 4.28584354e-05 9.05055343e-05 5.80806117e-05]
 [4.87330971e-05 1.75188310e-05 7.70476961e-05 2.31081419e-05]
 [9.36191817e-05 2.39585502e-05 4.72689511e-05 8.13528265e-05]
 [7.11039987e-05 1.21533450e-06 6.42774561e-05 4.04056178e-06]
 [3.13082875e-05 5.00994516e-06 8.24151909e-05 4.27704599e-05]
 [2.27963320e-05 9.02766672e-05 8.30633807e-05 4.77020252e-05]
 [8.65244722e-05 5.94387699e-05 3.70695158e-05 5.11285441e-05]
 [2.55819309e-07 8.52611625e-05 9.16777152e-05 3.83948928e-05]
 [4.26230700e-05 1.32936064e-05 7.08374175e-06 6.55420371e-05]
 [1.12536386e-05 8.18479891e-05 5.05042894e-05 4.95659251e-05]
 [7.24986804e-06 1.00747072e-05 1.08009854e-05 6.72335283e-05]
 [7.61759986e-05 8.25187839e-05 4.58926213e-05 3.81978849e-05]
 [2.33614486e-05 5.98126145e-05 1.

KeyboardInterrupt: 